In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os 
import urllib
import cPickle, pickle
import numpy as np
import shutil
import os
from operator import attrgetter

In [5]:
DATASET = 'salami' # or 'beatles'

In [7]:
if DATASET == 'salami':
    song_path = "../salami_subset/"
    song_list = [s[:-4] for s in os.listdir(song_path) if s.endswith('.mp3')]
    song_list.remove('1605')
    
    reference_clips = [
                    '1605/1605_20_0.wav', 
                    '1605/1605_20_2.wav', 
                    '1605/1605_20_4.wav',
                    '1605/1605_20_6.wav', 
                    '1605/1605_20_8.wav',
                  ]

else:
    song_path = "../beatles_subset/"
    song_list = [s[:-4] for s in os.listdir(song_path) if s.endswith('.wav')]
    song_list.remove('Come Together')
    
    reference_clips = [
                    'Come Together/Come Together_20_0.wav', 
                    'Come Together/Come Together_20_6.wav', 
                    'Come Together/Come Together_20_8.wav',
                    'Come Together/Come Together_20_10.wav', 
                    'Come Together/Come Together_20_14.wav',
                    'Come Together/Come Together_20_16.wav',
                    'Come Together/Come Together_20_18.wav',
                  ]
    

clips_path = "../clips/"
print song_list

In [13]:
song_list.remove('1605')

['1054', '1605', '1621', '1627', '979']


In [15]:
selected_clips = [[clips for clips in os.listdir(clips_path+clip_folder) if clips.endswith('.wav') 
                   and clips.split('_')[-2]=='20'] 
                  for clip_folder in os.listdir(clips_path) if clip_folder in song_list]

In [16]:
class clip:
    def __init__(self, name, duration, idx):
        self.name = name
        self.duration = int(duration)
        self.idx = int(idx)
    def __repr__(self):
        return self.name+"_"+str(self.duration)+"_"+str(self.idx)
    def path(self, path):
        return path+self.name+"/"+self.__repr__()
    def url(self, url_path="//dl.dropboxusercontent.com/u/4893373/amt_batch/"):
        return "https:"+urllib.quote(url_path+self.name+"/"+self.__repr__())

In [17]:
parsed_songs = []
for i, song in enumerate(selected_clips):
    parsed_songs.append([clip(*c[:-4].split("_")) for c in song])

In [18]:
batch_num = 2
non_overlap_dict = dict([(s, [[] for j in range(batch_num)]) for s in song_list])

In [19]:
for parsed_clips in parsed_songs:
    for c in parsed_clips:
        idx = (c.idx)%batch_num
        non_overlap_dict[c.name][idx].append(c)

In [21]:
test_batches = [[] for j in range(batch_num)]

for n, clip_list in non_overlap_dict.items():
    for i, clips in enumerate(clip_list):    
        test_batches[i].extend(clips)

In [22]:
for test_batch in test_batches:
    extra = 9-len(test_batch)%9
    test_batch.extend(np.random.choice(test_batch, size=extra, replace=False))
    print len(test_batch)%9

0
0


In [23]:
print len(test_batches[1])

36


In [25]:
reference_list = []
permute_indices = []
permute_batches = []
exp_song_dict=dict([(j,{})for j in range(batch_num)])
test_count = 1
for b_idx, test_batch in enumerate(test_batches):
    permute_idx = np.random.permutation(len(test_batch))
    permute_batch = [test_batch[i] for i in permute_idx]
    
    converted_song = []
    for i, song in enumerate(permute_batch):
#         idx = b_idx*10+test_count
        seg_name = "seg%02d_%02d.wav" % (test_count, i%9+1)
        shutil.copy(song.path(clips_path)+'.wav', '../amt_batch/'+seg_name)
        exp_song_dict[b_idx][seg_name]=song
        if i%9+1 == 1:
            ref = np.random.choice(reference_clips)
            shutil.copy(clips_path+ref, '../amt_batch/seg%02d_10.wav' % (test_count))
            exp_song_dict[b_idx]['seg%02d_10.wav' % (test_count)]=ref        
        if i%9+1 == 9:
            test_count+=1
        
    permute_indices.append(permute_idx)
    permute_batches.append(permute_batch)

f = open('amt_clip_04192017.dict', 'w')
pickle.dump(exp_song_dict, f)
f.close()

In [26]:
print len(exp_song_dict[1])

40
